In [1]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

from src.ADMM_MIP import *

import warnings
warnings.simplefilter('ignore')

In [2]:
mdl = QuadraticProgram('test Problem')

# binary variables
mdl.binary_var(name = 'v')
mdl.binary_var(name = 'w')
mdl.binary_var(name = 't')
mdl.continuous_var(name = 'u')

# objective function
mdl.minimize(
    linear={'v':1, 'w':1, 't':1, 'u':-20},
    quadratic={('u','u'):5},
    constant=20
    )

# le constraints
mdl.linear_constraint(linear={'t':1, 'u':1, 'v':1, 'w':2}, sense='<=', rhs=3)
mdl.linear_constraint(linear={'t':1, 'v':1, 'w':1}, sense='>=', rhs=1)
mdl.linear_constraint(linear={'v':1, 'w':1}, sense='==', rhs=1)

#print(mdl.export_as_lp_string())
print(mdl.prettyprint())

Problem name: test Problem

Minimize
  5*u^2 + t - 20*u + v + w + 20

Subject to
  Linear constraints (3)
    t + u + v + 2*w <= 3  'c0'
    t + v + w >= 1  'c1'
    v + w == 1  'c2'

  Continuous variables (1)
    0 <= u

  Binary variables (3)
    v w t



In [3]:
ADMMxClassical = ADMMxClassical_Optimizer(mdl)

n_bin_vars = ADMMxClassical.n_bin_vars
n_cont_vars = ADMMxClassical.n_cont_vars
n_eq = ADMMxClassical.n_eq
n_le = ADMMxClassical.n_le
n_ge = ADMMxClassical.n_ge

print(
        '-'*50,
        '[Problem Info]',
        '-Decision Variables',
        f'   number of binary vars: {n_bin_vars}',
        f'   number of continuous vars: {n_cont_vars}',
        '-Linear Constraints',
        f'   number of eq constraints: {n_eq}',
        f'   number of le constraints: {n_le}',
        f'   number of ge constraints: {n_ge}',
        '-'*50,
        sep='\n'
    )

--------------------------------------------------
[Problem Info]
-Decision Variables
   number of binary vars: 3
   number of continuous vars: 1
-Linear Constraints
   number of eq constraints: 1
   number of le constraints: 1
   number of ge constraints: 1
--------------------------------------------------


In [4]:
cplex_result = CplexOptimizer().solve(mdl)
opt_val = cplex_result.fval
x_sol = cplex_result.x

n_bin = mdl.get_num_binary_vars()

print(
        '-'*80,
        '[CPLEX Result]',
        f'   optimal value: {cplex_result.fval:.3f}',
        f'   optimal solution (binary): {x_sol[:n_bin].astype(int)}',
        f'   optimal solution (continuous): {x_sol[n_bin:]}',
        f'   status: {cplex_result.status.name}',
        '-'*80,
        sep='\n'
    )

--------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: 1.000
   optimal solution (binary): [1 0 0]
   optimal solution (continuous): [2.]
   status: SUCCESS
--------------------------------------------------------------------------------


In [5]:
ADMMxClassical = ADMMxClassical_Optimizer(mdl)

step_max = 5
n_lin_const = ADMMxClassical.n_lin_const
x_ini = np.array([0.5]*(n_bin_vars+n_cont_vars))
lam_ini = np.full(n_lin_const, 30.)
rho = 10.

admm_result = ADMMxClassical.solve(step_max, x_ini, lam_ini, rho)
x_sol = admm_result.x
n_bin = mdl.get_num_binary_vars()

pd.options.display.float_format = '{:.3f}'.format
print(
        '-'*100,
        '[ADMMxClassic Result]',
        f'   optimal value: {admm_result.fval:.3f}',
        f'   optimal solution (binary): {x_sol[:n_bin].astype(int)}',
        f'   optimal solution (continuous): {x_sol[n_bin:]}',
        f'   status: {admm_result.status.name}',
        '-'*100,
        ADMMxClassical.opt_record,
        '-'*100,
        sep='\n'
    )

----------------------------------------------------------------------------------------------------
[ADMMxClassic Result]
   optimal value: 1.000
   optimal solution (binary): [1 0 0]
   optimal solution (continuous): [2.]
   status: SUCCESS
----------------------------------------------------------------------------------------------------
        objective function value      status
step 0                    12.750    feasible
step 1                    23.000  infeasible
step 2                     1.000    feasible
step 3                     1.000    feasible
step 4                     1.000    feasible
step 5                     1.000    feasible
----------------------------------------------------------------------------------------------------


In [6]:
admm_params = ADMMParameters(
    rho_initial=10, beta=10, factor_c=10, maxiter=100, three_block=True, tol=1.0e-6
)

# initialize ADMM with classical QUBO and convex optimizer
admm = ADMMOptimizer(
    params=admm_params, qubo_optimizer=CplexOptimizer(), continuous_optimizer=CplexOptimizer()
)

# run ADMM to solve problem
admm3b_result = admm.solve(mdl)
print(admm3b_result.prettyprint())

data = admm3b_result.state.residuals

print('-'*80)
for step in range(len(data)):
    print(f'step = {step} \t err = {data[step]}')

objective function value: 1.0000000653442171
variable values: v=1.0, w=0.0, t=0.0, u=1.9998856809583552
status: SUCCESS
--------------------------------------------------------------------------------
step = 0 	 err = 0.3000000002278503
step = 1 	 err = 0.1398746041825084
step = 2 	 err = 0.06329167692740957
step = 3 	 err = 0.027814594234713597
step = 4 	 err = 0.01193917685356103
step = 5 	 err = 0.005068004179429022
step = 6 	 err = 0.0021697144789762273
step = 7 	 err = 0.0009595987670665706
step = 8 	 err = 0.0004392471688310234
step = 9 	 err = 0.0002091842727325028
step = 10 	 err = 0.00010536332589212367
step = 11 	 err = 5.6096447456250464e-05
step = 12 	 err = 3.0724517044419045e-05
step = 13 	 err = 1.7414267732463962e-05
step = 14 	 err = 1.0266070189868044e-05
step = 15 	 err = 6.2124832877653335e-06
step = 16 	 err = 3.8082205487687576e-06
step = 17 	 err = 2.2587755806248204e-06
step = 18 	 err = 1.2950010579788537e-06
step = 19 	 err = 7.290760203947967e-07
